In [5]:
import yfinance as yf
import pandas as pd
import numpy as np  # Import numpy

# Step 1: Download maximum available OHLC data
ticker = '63MOONS.NS'  # You can change this to any ticker you like
interval = '1d'  # Daily data

print(f"Downloading historical data for {ticker}...")
try:
    # Use progress=False for cleaner output if not running interactively
    data = yf.download(ticker, period='max', interval=interval, progress=False)
    print("Download complete.")

    if data.empty:
        print(f"No data downloaded for {ticker}. Cannot perform analysis.")
    else:
        print(f"Data range: {data.index.min().date()} to {data.index.max().date()}")
        print(f"Total records downloaded: {len(data)}")

        # Step 2: Create a new column 'Close_vs_Open' using numpy.select
        # This is generally faster and more robust than apply(axis=1)
        conditions = [
            data['Close'] > data['Open'],  # Condition for 1
            data['Open'] > data['Close']   # Condition for -1 (equivalent to Close < Open)
        ]
        choices = [1, -1]
        # np.select applies choices based on conditions, default is 0 if neither condition is met
        data['Close_vs_Open'] = np.select(conditions, choices, default=0)

        # Step 3: Calculate counts and ratio
        count_pos = (data['Close_vs_Open'] == 1).sum()
        count_neg = (data['Close_vs_Open'] == -1).sum()
        count_zero = (data['Close_vs_Open'] == 0).sum() # Optional: count days where Open == Close

        # Handle potential division by zero for the ratio
        if count_pos > 0:
            ratio_neg_to_pos = count_neg / count_pos
        else:
            # Assign NaN or infinity if there are no positive days
            ratio_neg_to_pos = np.nan if count_neg == 0 else float('inf')

        # Step 4: Print results
        print("\n--- Analysis Results ---")
        print(f"Number of days Close > Open (1): {count_pos}")
        print(f"Number of days Close < Open (-1): {count_neg}")
        print(f"Number of days Close == Open (0): {count_zero}")
        print(f"Total trading days analyzed: {count_pos + count_neg + count_zero}")
        # Ensure the ratio format handles potential NaN or inf
        if np.isnan(ratio_neg_to_pos):
             print("Ratio (-1s to 1s): Undefined (no days with Close > Open)")
        elif np.isinf(ratio_neg_to_pos):
             print("Ratio (-1s to 1s): Infinity (only days with Close < Open or Close == Open)")
        else:
            print(f"Ratio (-1s to 1s): {ratio_neg_to_pos:.4f}") # Using 4 decimal places for more precision

except Exception as e:
    print(f"\nAn error occurred: {e}")
    print("Please ensure the 'yfinance' and 'numpy' libraries are installed (`pip install yfinance numpy`)")
    print("Also check your internet connection and the ticker symbol.")


Download complete.
Data range: 2005-06-20 to 2025-04-30
Total records downloaded: 4899

--- Analysis Results ---
Number of days Close > Open (1): 1920
Number of days Close < Open (-1): 2848
Number of days Close == Open (0): 131
Total trading days analyzed: 4899
Ratio (-1s to 1s): 1.4833


In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
import time
import warnings

# --- Configuration ---
# Remove duplicates and ensure list type
tickers_raw = ['3MINDIA.NS', '63MOONS.NS', 'AARTIDRUGS.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ACC.NS', 'ACE.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ADSL.NS', 'ADVENZYMES.NS', 'AFFLE.NS', 'AHLUCONT.NS', 'AIAENG.NS', 'AJANTPHARM.NS', 'AJMERA.NS', 'ALEMBICLTD.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALLCARGO.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANTRAJ.NS', 'ANUP.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLO.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'ARTEMISMED.NS', 'ARVIND.NS', 'ARVINDFASN.NS', 'ASAHIINDIA.NS', 'ASALCBR.NS', 'ASHAPURMIN.NS', 'ASHOKA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTEC.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAMICRO.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATUL.NS', 'AUBANK.NS', 'AURIONPRO.NS', 'AUROPHARMA.NS', 'AVANTIFEED.NS', 'AXISBANK.NS', 'AXISCADES.NS', 'BAJAJ-AUTO.NS', 'BAJAJELEC.NS', 'BAJAJFINSV.NS', 'BAJAJHIND.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALAMINES.NS', 'BALKRISIND.NS', 'BALMLAWRIE.NS', 'BALPHARMA.NS', 'BALRAMCHIN.NS', 'BANCOINDIA.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBL.NS', 'BBTC.NS', 'BDL.NS', 'BECTORFOOD.NS', 'BEL.NS', 'BEML.NS', 'BEPL.NS', 'BERGEPAINT.NS', 'BFUTILITIE.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHEL.NS', 'BIOCON.NS', 'BIRLACORPN.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BOMDYEING.NS', 'BORORENEW.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMLINFINE.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPACITE.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CARERATING.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIGNITITEC.NS', 'CIPLA.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COLPAL.NS', 'CONCOR.NS', 'COROMANDEL.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CSBBANK.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAMATICS.NS', 'DBL.NS', 'DBREALTY.NS', 'DCAL.NS', 'DCBBANK.NS', 'DCMSHRIRAM.NS', 'DCW.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELTACORP.NS', 'DHAMPURSUG.NS', 'DHANI.NS', 'DISHTV.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DLINKINDIA.NS', 'DMART.NS', 'DONEAR.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EDELWEISS.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'EKC.NS', 'ELECON.NS', 'ELECTCAST.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'EPL.NS', 'EQUITASBNK.NS', 'ERIS.NS', 'ESCORTS.NS', 'EXCELINDUS.NS', 'EXIDEIND.NS', 'FACT.NS', 'FCL.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINEORG.NS', 'FINPIPE.NS', 'FLUOROCHEM.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'FSL.NS', 'GABRIEL.NS', 'GAEL.NS', 'GAIL.NS', 'GALLANTT.NS', 'GANECOS.NS', 'GANESHHOUC.NS', 'GARFIBRES.NS', 'GENESYS.NS', 'GENUSPOWER.NS', 'GEOJITFSL.NS', 'GEPIL.NS', 'GESHIP.NS', 'GHCL.NS', 'GICHSGFIN.NS', 'GICRE.NS', 'GILLETTE.NS', 'GIPCL.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GLOBUSSPR.NS', 'GMDCLTD.NS', 'GMMPFAUDLR.NS', 'GNFC.NS', 'GODFRYPHLP.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GOKEX.NS', 'GOLDIAM.NS', 'GOODLUCK.NS', 'GPIL.NS', 'GPPL.NS', 'GPTINFRA.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GREAVESCOT.NS', 'GRSE.NS', 'GSFC.NS', 'GSPL.NS', 'GTLINFRA.NS', 'GUJALKALI.NS', 'GUJGASLTD.NS', 'GULFOILLUB.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HATHWAY.NS', 'HAVELLS.NS', 'HCC.NS', 'HCG.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEMIPROP.NS', 'HERITGFOOD.NS', 'HEROMOTOCO.NS', 'HESTERBIO.NS', 'HFCL.NS', 'HGINFRA.NS', 'HIKAL.NS', 'HIL.NS', 'HIMATSEIDE.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDOILEXP.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HITECH.NS', 'HONAUT.NS', 'HPL.NS', 'HSCL.NS', 'HUBTOWN.NS', 'HUDCO.NS', 'IBREALEST.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'ICIL.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFBIND.NS', 'IFCI.NS', 'IGARASHI.NS', 'IGL.NS', 'IIFL.NS', 'IMAGICAA.NS', 'IMFA.NS', 'INDHOTEL.NS', 'INDIAGLYCO.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIANHUME.NS', 'INDIGO.NS', 'INDOTECH.NS', 'INDRAMEDCO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFIBEAM.NS', 'INFY.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IOLCP.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'ITC.NS', 'ITDCEM.NS', 'J&KBANK.NS', 'JAIBALAJI.NS', 'JAICORPLTD.NS', 'JAMNAAUTO.NS', 'JASH.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JCHAC.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JINDRILL.NS', 'JINDWORLD.NS', 'JISLJALEQS.NS', 'JKCEMENT.NS', 'JKIL.NS', 'JKLAKSHMI.NS', 'JKPAPER.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPASSOCIAT.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUSTDIAL.NS', 'JYOTHYLAB.NS', 'JYOTISTRUC.NS', 'KABRAEXTRU.NS', 'KAJARIACER.NS', 'KAMATHOTEL.NS', 'KAMDHENU.NS', 'KARURVYSYA.NS', 'KCP.NS', 'KDDL.NS', 'KEC.NS', 'KEI.NS', 'KELLTONTEC.NS', 'KERNEX.NS', 'KIRIINDUS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KITEX.NS', 'KNRCON.NS', 'KOTAKBANK.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'KRBL.NS', 'KSB.NS', 'KSCL.NS', 'KTKBANK.NS', 'LALPATHLAB.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LINDEINDIA.NS', 'LT.NS', 'LTTS.NS', 'LUPIN.NS', 'LUXIND.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHLIFE.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANGCHEFER.NS', 'MANGLMCEM.NS', 'MANINDS.NS', 'MANINFRA.NS', 'MARICO.NS', 'MARINE.NS', 'MARKSANS.NS', 'MARUTI.NS', 'MASTEK.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MIDHANI.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOIL.NS', 'MOREPENLAB.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MPSLTD.NS', 'MRF.NS', 'MRPL.NS', 'MSTCLTD.NS', 'MTNL.NS', 'MUTHOOTFIN.NS', 'NACLIND.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVINFLUOR.NS', 'NAVKARCORP.NS', 'NBCC.NS', 'NCC.NS', 'NECLIFE.NS', 'NELCO.NS', 'NEOGEN.NS', 'NESCO.NS', 'NESTLEIND.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NFL.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIITLTD.NS', 'NITINSPIN.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NOCIL.NS', 'NSIL.NS', 'NTPC.NS', 'OAL.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLECTRA.NS', 'ONGC.NS', 'OPTIEMUS.NS', 'ORIENTCEM.NS', 'ORIENTELEC.NS', 'ORISSAMINE.NS', 'PAGEIND.NS', 'PAISALO.NS', 'PANACEABIO.NS', 'PARACABLES.NS', 'PARAGMILK.NS', 'PATELENG.NS', 'PCJEWELLER.NS', 'PEL.NS', 'PENIND.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PGHH.NS', 'PGIL.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PITTIENG.NS', 'PNB.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS', 'POKARNA.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POLYPLEX.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'POWERMECH.NS', 'PRAJIND.NS', 'PRAKASH.NS', 'PREMEXPLN.NS', 'PRESTIGE.NS', 'PRICOLLTD.NS', 'PRINCEPIPE.NS', 'PRIVISCL.NS', 'PSB.NS', 'PTC.NS', 'QUESS.NS', 'QUICKHEAL.NS', 'RADICO.NS', 'RAIN.NS', 'RAJESHEXPO.NS', 'RALLIS.NS', 'RAMCOCEM.NS', 'RATNAMANI.NS', 'RAYMOND.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'REFEX.NS', 'RELIANCE.NS', 'RELIGARE.NS', 'RELINFRA.NS', 'RENUKA.NS', 'REPCOHOME.NS', 'RESPONIND.NS', 'RGL.NS', 'RIIL.NS', 'RITES.NS', 'RKFORGE.NS', 'ROHLTD.NS', 'ROUTE.NS', 'RPOWER.NS', 'RTNPOWER.NS', 'RVNL.NS', 'SAIL.NS', 'SALZERELEC.NS', 'SANGHVIMOV.NS', 'SANOFI.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SDBL.NS', 'SEQUENT.NS', 'SHAKTIPUMP.NS', 'SHALBY.NS', 'SHARDACROP.NS', 'SHARDAMOTR.NS', 'SHAREINDIA.NS', 'SHILPAMED.NS', 'SHK.NS', 'SHREECEM.NS', 'SHRIPISTON.NS', 'SIYSIL.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SKIPPER.NS', 'SMLISUZU.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONATSOFTW.NS', 'SOUTHBANK.NS', 'SPAL.NS', 'SPANDANA.NS', 'SPARC.NS', 'SPMLINFRA.NS', 'SRF.NS', 'STAR.NS', 'STARCEMENT.NS', 'STLTECH.NS', 'SUBEXLTD.NS', 'SUDARSCHEM.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNFLAG.NS', 'SUNPHARMA.NS', 'SUNTECK.NS', 'SUPREMEIND.NS', 'SURYAROSNI.NS', 'SUVEN.NS', 'SUVENPHAR.NS', 'SUZLON.NS', 'SWANENERGY.NS', 'SWARAJENG.NS', 'SWSOLAR.NS', 'SYMPHONY.NS', 'SYNGENE.NS', 'TAJGVK.NS', 'TANLA.NS', 'TARC.NS', 'TASTYBITE.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TBZ.NS', 'TCPLPACK.NS', 'TCS.NS', 'TDPOWERSYS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'TEXINFRA.NS', 'TEXRAIL.NS', 'TFCILTD.NS', 'THANGAMAYL.NS', 'THEMISMED.NS', 'THERMAX.NS', 'THOMASCOOK.NS', 'TI.NS', 'TIINDIA.NS', 'TIMETECHNO.NS', 'TIMKEN.NS', 'TIRUMALCHM.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNICHEMLAB.NS', 'UNIONBANK.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'V2RETAIL.NS', 'VADILALIND.NS', 'VAIBHAVGBL.NS', 'VAKRANGEE.NS', 'VALIANTORG.NS', 'VARROC.NS', 'VBL.NS', 'VEDL.NS', 'VESUVIUS.NS', 'VGUARD.NS', 'VIMTALABS.NS', 'VINATIORGA.NS', 'VINDHYATEL.NS', 'VIPCLOTHNG.NS', 'VIPIND.NS', 'VISHNU.NS', 'VMART.NS', 'VOLTAMP.NS', 'VOLTAS.NS', 'VSTIND.NS', 'VTL.NS', 'WABAG.NS', 'WALCHANNAG.NS', 'WEBELSOLAR.NS', 'WELCORP.NS', 'WELENT.NS', 'WHIRLPOOL.NS', 'WOCKPHARMA.NS', 'WSTCSTPAPR.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ABSLAMC.NS']
tickers = sorted(list(set(tickers_raw))) # Remove duplicates and sort

interval = '1d'
period = 'max'
batch_size = 50
max_retries = 3
retry_delay = 5  # seconds
output_file = 'open_vs_close_analysis.csv'

# Store results and failures
results = []
failed_tickers = [] # Tickers requested but data couldn't be processed

# Suppress specific yfinance warnings if desired (optional)
# warnings.filterwarnings("ignore", message=".*Converting timezone from.*")
# warnings.filterwarnings("ignore", message=".*No data found for ticker.*") # yfinance handles this internally often

# --- Analysis Function ---
def analyze_ticker_data(ticker_symbol, open_prices, close_prices):
    """Analyzes open vs close prices for a single ticker."""
    # Ensure inputs are Series and drop NaNs based on both
    combined = pd.DataFrame({'Open': open_prices, 'Close': close_prices}).dropna()

    # Check if data remains after dropping NaNs
    if combined.empty:
        print(f"    - No valid data after dropna for {ticker_symbol}")
        # Don't add to failed_tickers here, as it might have downloaded, just no usable data.
        # The main loop will catch tickers that failed download entirely.
        return

    # Calculate comparison: 1 if Close > Open, -1 if Close < Open, 0 otherwise
    conditions = [
        combined['Close'] > combined['Open'],
        combined['Open'] > combined['Close']
    ]
    choices = [1, -1]
    close_vs_open = np.select(conditions, choices, default=0)

    # Calculate counts
    count_pos = (close_vs_open == 1).sum()
    count_neg = (close_vs_open == -1).sum()
    count_zero = (close_vs_open == 0).sum()
    total_days = len(close_vs_open) # Use length of comparison array

    # Calculate ratio, handling division by zero and inf cases
    if count_pos > 0:
        ratio_neg_to_pos = count_neg / count_pos
    elif count_neg > 0: # count_pos is 0, but count_neg is not
        ratio_neg_to_pos = float('inf')
    else: # Both count_pos and count_neg are 0
        ratio_neg_to_pos = np.nan

    # Format ratio for output
    if np.isnan(ratio_neg_to_pos):
        ratio_str = None
    elif np.isinf(ratio_neg_to_pos):
        ratio_str = 'inf'
    else:
        ratio_str = round(ratio_neg_to_pos, 4)

    # Append results
    results.append({
        'Ticker': ticker_symbol,
        'Start Date': combined.index.min().date(),
        'End Date': combined.index.max().date(),
        'Days Close > Open': count_pos,
        'Days Close < Open': count_neg,
        'Days Close == Open': count_zero,
        'Total Days': total_days,
        'Ratio (-1s to 1s)': ratio_str
    })
    # print(f"    + Successfully analyzed {ticker_symbol}") # Optional: uncomment for verbose output

# --- Batch Downloader with Retry ---
def download_batch(batch):
    """Downloads a batch of tickers with retries."""
    for attempt in range(1, max_retries + 1):
        try:
            print(f"\nAttempt {attempt}/{max_retries} downloading batch: {batch}")
            # Use default yfinance behavior (no group_by) - returns MultiIndex columns ('Open'/'Ticker', 'Close'/'Ticker')
            # threads=True is often beneficial for batch downloads
            data = yf.download(
                tickers=batch,
                period=period,
                interval=interval,
                progress=False, # Show progress per ticker in batch
                threads=True    # Use threading
            )
            # Check if *any* data was returned (even if some tickers failed)
            if not data.empty:
                return data
            else:
                # yfinance might return an empty DataFrame if all tickers in the batch fail
                print(f"Attempt {attempt} resulted in empty DataFrame for batch {batch}")

        except Exception as e:
            print(f"Attempt {attempt} failed for batch {batch} with exception: {e}")

        # Wait before retrying if not the last attempt
        if attempt < max_retries:
            print(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
        else:
            print(f"Max retries reached for batch {batch}. Marking as failed.")

    return None # Return None if all attempts failed

# --- Main Execution ---
print(f"Starting analysis for {len(tickers)} unique tickers...")
processed_tickers = set() # Keep track of tickers successfully processed

for i in range(0, len(tickers), batch_size):
    batch = tickers[i:i + batch_size]
    data = download_batch(batch)

    if data is None or data.empty:
        print(f"-> Batch failed entirely or returned empty data: {batch}")
        # Add all tickers from this *requested* batch to failed_tickers
        # Only add those not already processed/failed previously
        failed_tickers.extend([t for t in batch if t not in processed_tickers and t not in failed_tickers])
        continue # Move to the next batch

    print(f"Processing downloaded data for batch: {batch}")

    # Identify tickers actually present in the downloaded data's columns
    # Default yf.download puts Ticker in level 1 of MultiIndex
    if isinstance(data.columns, pd.MultiIndex):
        tickers_in_data = data.columns.get_level_values(1).unique()
    # Handle case where only one ticker was requested *and* downloaded successfully
    elif isinstance(data.columns, pd.Index) and len(batch) == 1:
         # If only one ticker was in the batch, yfinance returns a simple Index
         # The columns will be 'Open', 'Close', etc.
         # We reconstruct the expected structure for analysis function compatibility
         ticker = batch[0]
         if 'Open' in data.columns and 'Close' in data.columns:
             analyze_ticker_data(ticker, data['Open'], data['Close'])
             processed_tickers.add(ticker)
         else:
            print(f"    - Missing 'Open' or 'Close' column for single ticker {ticker}")
            if ticker not in processed_tickers: failed_tickers.append(ticker)
         continue # Skip the MultiIndex processing for this single ticker case
    else:
        # Unexpected data structure
        print(f"-> Unexpected data structure received for batch: {batch}")
        failed_tickers.extend([t for t in batch if t not in processed_tickers and t not in failed_tickers])
        continue

    # Process tickers found in the MultiIndex data
    for ticker in tickers_in_data:
        # Check if both Open and Close data are available for this ticker
        open_col = ('Open', ticker)
        close_col = ('Close', ticker)

        if open_col in data.columns and close_col in data.columns:
            try:
                analyze_ticker_data(ticker, data[open_col], data[close_col])
                processed_tickers.add(ticker) # Mark as processed successfully
            except Exception as e:
                print(f"    ! Error analyzing {ticker}: {e}")
                if ticker not in processed_tickers: failed_tickers.append(ticker)
        else:
            # This ticker was in the data, but lacked Open or Close columns
            print(f"    - Missing 'Open' or 'Close' data for {ticker} within the downloaded batch.")
            if ticker not in processed_tickers: failed_tickers.append(ticker)

    # Check which tickers from the original batch were *not* successfully processed
    for ticker in batch:
        if ticker not in processed_tickers and ticker not in failed_tickers:
            # This ticker was requested but wasn't in tickers_in_data or failed analysis silently
             print(f"    - Ticker {ticker} requested but not found/processed in batch result.")
             failed_tickers.append(ticker)


# --- Export Results to CSV ---
if results:
    print(f"\n--- Saving {len(results)} analysis results to {output_file} ---")
    df_results = pd.DataFrame(results)
    # Sort results for consistency if desired
    df_results.sort_values(by='Ticker', inplace=True)
    try:
        df_results.to_csv(output_file, index=False)
        print("    Save successful.")
    except Exception as e:
        print(f"    Error saving results CSV: {e}")
else:
    print("\n--- No results generated to save. ---")


# --- Save Failed Tickers ---
# Ensure failed_tickers list contains unique entries before saving
unique_failed_tickers = sorted(list(set(failed_tickers)))
if unique_failed_tickers:
    failed_file = 'failed_tickers.csv'
    print(f"--- Saving {len(unique_failed_tickers)} tickers with issues to {failed_file} ---")
    try:
        pd.DataFrame(unique_failed_tickers, columns=['Ticker']).to_csv(failed_file, index=False)
        print("    Save successful.")
    except Exception as e:
        print(f"    Error saving failed tickers CSV: {e}")
else:
    print("--- No tickers failed processing. ---")

print("\n--- Script finished ---")

Starting analysis for 649 unique tickers...

Attempt 1/3 downloading batch: ['3MINDIA.NS', '63MOONS.NS', 'AARTIDRUGS.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ADSL.NS', 'ADVENZYMES.NS', 'AFFLE.NS', 'AHLUCONT.NS', 'AIAENG.NS', 'AJANTPHARM.NS', 'AJMERA.NS', 'ALEMBICLTD.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALLCARGO.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANTRAJ.NS', 'ANUP.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLO.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'ARTEMISMED.NS', 'ARVIND.NS', 'ARVINDFASN.NS', 'ASAHIINDIA.NS', 'ASALCBR.NS', 'ASHAPURMIN.NS', 'ASHOKA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTEC.NS', 'ASTERDM.NS', 'ASTRAL.NS']



1 Failed download:
['AFFLE.NS']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Processing downloaded data for batch: ['3MINDIA.NS', '63MOONS.NS', 'AARTIDRUGS.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ADSL.NS', 'ADVENZYMES.NS', 'AFFLE.NS', 'AHLUCONT.NS', 'AIAENG.NS', 'AJANTPHARM.NS', 'AJMERA.NS', 'ALEMBICLTD.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALLCARGO.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANTRAJ.NS', 'ANUP.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLO.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'ARTEMISMED.NS', 'ARVIND.NS', 'ARVINDFASN.NS', 'ASAHIINDIA.NS', 'ASALCBR.NS', 'ASHAPURMIN.NS', 'ASHOKA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTEC.NS', 'ASTERDM.NS', 'ASTRAL.NS']
    - No valid data after dropna for AFFLE.NS

Attempt 1/3 downloading batch: ['ASTRAMICRO.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATUL.NS', 'AUBANK.NS', 'AURIONPRO.NS', 'AUROPHARMA.NS', 'AVANTIFEED.NS', 'AXISBANK.NS', 'AXISCADES.NS', 'BAJAJ-AU


1 Failed download:
['NMDC.NS']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Processing downloaded data for batch: ['MRPL.NS', 'MSTCLTD.NS', 'MTNL.NS', 'MUTHOOTFIN.NS', 'NACLIND.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVINFLUOR.NS', 'NAVKARCORP.NS', 'NBCC.NS', 'NCC.NS', 'NECLIFE.NS', 'NELCO.NS', 'NEOGEN.NS', 'NESCO.NS', 'NESTLEIND.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NFL.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIITLTD.NS', 'NITINSPIN.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NOCIL.NS', 'NSIL.NS', 'NTPC.NS', 'OAL.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLECTRA.NS', 'ONGC.NS', 'OPTIEMUS.NS', 'ORIENTCEM.NS', 'ORIENTELEC.NS', 'ORISSAMINE.NS', 'PAGEIND.NS', 'PAISALO.NS', 'PANACEABIO.NS', 'PARACABLES.NS', 'PARAGMILK.NS', 'PATELENG.NS', 'PCJEWELLER.NS', 'PEL.NS']
    - No valid data after dropna for NMDC.NS

Attempt 1/3 downloading batch: ['PENIND.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PGHH.NS', 'PGIL.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PITTIENG.NS', 'PNB.NS', 'PNBHOUSING.NS', 'PNCIN

In [13]:
import yfinance as yf
import pandas as pd

tickers = ['RELIANCE.NS', 'HDFCBANK.NS', 'BAJFINANCE.NS', 'SBIN.NS', 'ICICIBANK.NS']

start_date = "2013-01-15"
end_date = "2018-01-15"

result = pd.DataFrame()

for ticker in tickers:
    df = yf.download(ticker, start=start_date, end=end_date, interval="1d")
    if not df.empty:
        df[ticker] = ((df['Open'] - df['Close']) / df['Open'])
        result = pd.concat([result, df[[ticker]]], axis=1)

result['Sum_Per_Date'] = result.sum(axis=1)

result['Modified_Sum'] = result['Sum_Per_Date'].apply(lambda x: -0.1 if x < -0.1 else x)

result.to_csv('ticker_open_close_diff.csv')

total_sum_original = result['Sum_Per_Date'].sum()
total_sum_modified = result['Modified_Sum'].sum()

print("\nTotal sum of 'Sum_Per_Date':", total_sum_original)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Total sum of 'Sum_Per_Date': 3.8353226513703174


In [16]:
import yfinance as yf
import pandas as pd
import random
from datetime import datetime, timedelta
from tqdm import tqdm

tickers = ['MRF.NS', 'PAGEIND.NS', 'HONAUT.NS', '3MINDIA.NS', 'ABBOTINDIA.NS']

random_dates = []
for _ in range(100):
    year = random.randint(2010, 2024)
    month = random.randint(1, 4)
    day = random.randint(1, 11)
    start = datetime(year, month, day)
    end = start + timedelta(days=365)
    random_dates.append((start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")))

summary_results = []

for i, (start_date, end_date) in enumerate(tqdm(random_dates, desc="Processing 1000 random date ranges")):
    result = pd.DataFrame()
    for ticker in tickers:
        df = yf.download(ticker, start=start_date, end=end_date, interval="1d", progress=False)
        if not df.empty:
            df[ticker] = (df['Open'] - df['Close']) / df['Open']
            result = pd.concat([result, df[[ticker]]], axis=1)

    if not result.empty:
        result['Sum_Per_Date'] = result.sum(axis=1)
        result['Modified_Sum'] = result['Sum_Per_Date'].apply(lambda x: -0.1 if x < -0.1 else x)
        total_sum_original = result['Sum_Per_Date'].sum()
        total_sum_modified = result['Modified_Sum'].sum()
    else:
        total_sum_original = None
        total_sum_modified = None

    summary_results.append({
        "Iteration": i + 1,
        "Start Date": start_date,
        "End Date": end_date,
        "Original Sum": total_sum_original,
        "Modified Sum": total_sum_modified
    })

summary_df = pd.DataFrame(summary_results)
summary_df.to_csv("summary_of_100_runs.csv", index=False)

print(summary_df.head())

Processing 1000 random date ranges: 100%|██████████| 100/100 [01:35<00:00,  1.05it/s]

   Iteration  Start Date    End Date  Original Sum  Modified Sum
0          1  2016-01-01  2016-12-31      2.476427      2.731579
1          2  2015-03-03  2016-03-02      2.482814      2.582820
2          3  2010-04-10  2011-04-10      1.851145      2.575075
3          4  2023-02-07  2024-02-07      1.141107      1.214290
4          5  2013-04-11  2014-04-11      0.007856      0.007856


In [34]:
# Ensure yfinance is installed. Uncomment the next line if running in an environment
# where it might not be installed (like Google Colab or a fresh virtual environment)
# !pip install yfinance tenacity pandas

import yfinance as yf
import pandas as pd
import os
import time
from tenacity import retry, wait_fixed, stop_after_attempt

# === CONFIGURATION ===
tickers = ['3MINDIA.NS'] # Add more tickers as needed e.g., ['RELIANCE.NS', 'TCS.NS']

start_date = "2025-01-01"
end_date = "2025-04-26" # yfinance end_date is exclusive for intraday data
interval = "1h" # Fetching hourly data is necessary to find the lowest hour
batch_size = 50 # Process tickers in batches if you have many

# === CREATE FOLDER FOR OUTPUT ===
folder_name = "daily_low_times_data" # Changed folder name for clarity
os.makedirs(folder_name, exist_ok=True)
print(f"Daily low time data will be saved in folder: '{folder_name}'")

# === RETRY LOGIC FOR DOWNLOADING EACH TICKER ===
@retry(wait=wait_fixed(3), stop=stop_after_attempt(3)) # Retry 3 times with 3 sec wait
def download_ticker(ticker):
    """Downloads historical data for a given ticker with retries."""
    print(f"Downloading hourly data for: {ticker}")
    df = yf.download(
        ticker,
        start=start_date,
        end=end_date,
        interval=interval,
        progress=False, # Suppress yfinance download progress bar
        # auto_adjust=True # Optional: Use adjusted prices
        # prepost=True # Optional: Include pre/post market data if needed and available
    )
    # Check if 'Low' column exists, handle potential issues with downloaded data structure
    if df.empty or 'Low' not in df.columns:
        print(f"⚠️ Warning: DataFrame is empty or missing 'Low' column for {ticker}.")
        return pd.DataFrame() # Return empty DataFrame to avoid errors later
    return df

# === PROCESS IN BATCHES ===
print(f"Processing {len(tickers)} ticker(s) from {start_date} to {end_date} at {interval} interval.")

for i in range(0, len(tickers), batch_size):
    batch = tickers[i:i + batch_size]
    # Calculate total batches correctly
    total_batches = (len(tickers) + batch_size - 1) // batch_size
    print(f"\n📦 Processing batch {i // batch_size + 1} of {total_batches}")

    for ticker in batch:
        try:
            # Download hourly data for the ticker
            df_hourly = download_ticker(ticker)

            # Proceed only if data was successfully downloaded and is not empty
            if not df_hourly.empty:
                print(f"   Processing daily lows for {ticker}...")

                # --- CORE LOGIC FOR DAILY LOWS ---
                # Ensure the index is a DatetimeIndex
                df_hourly.index = pd.to_datetime(df_hourly.index)

                # Create a 'Date' column from the index for grouping
                # Use .tz_convert(None) if index is timezone-aware, then .date
                if df_hourly.index.tz is not None:
                     df_hourly['Date'] = df_hourly.index.tz_convert(None).date
                else:
                     df_hourly['Date'] = df_hourly.index.date


                # Find the index (timestamp) of the minimum 'Low' for each 'Date'
                # Using idxmin() on the grouped object gives the index label (Timestamp) of the min value within each group
                idx_daily_low = df_hourly.groupby('Date')['Low'].idxmin()

                # Select the rows corresponding to the daily lows using the obtained index
                daily_low_data = df_hourly.loc[idx_daily_low]

                # Refine the DataFrame to keep only the low price and add the timestamp as a column
                # The index of daily_low_data is already the timestamp of the low
                daily_low_summary = daily_low_data[['Low']].copy() # Select 'Low' column
                daily_low_summary.rename(columns={'Low': 'Daily Low Price'}, inplace=True)
                daily_low_summary['Timestamp of Low'] = daily_low_summary.index # Add timestamp column
                daily_low_summary.index.name = 'Date' # Set the index name to 'Date'

                # --- END CORE LOGIC ---

                # Save the daily low times summary to a CSV file
                file_path = os.path.join(folder_name, f"{ticker}_daily_low_times.csv")
                # Save with Date as index, Timestamp and Low Price as columns
                daily_low_summary.to_csv(file_path)
                print(f"✅ Saved daily low times for {ticker} to {file_path}")

            else:
                # Handle cases where yfinance returned no data (or data missing 'Low')
                print(f"📉 No valid hourly data found for {ticker} in the specified period to calculate daily lows.")

        except Exception as e:
            # Catch errors during download or processing for a specific ticker
            # Provide more specific error context if possible
            import traceback
            print(f"❌ Error processing {ticker}: {e}")
            # print(traceback.format_exc()) # Uncomment for detailed traceback if needed

    # Optional sleep between batches to avoid rate-limiting
    if len(tickers) > batch_size and i + batch_size < len(tickers):
        print(f"\n--- Pausing for 5 seconds before next batch ---")
        time.sleep(5)

print(f"\n🏁 Done! Daily low times CSV files saved in folder: '{folder_name}'")


Daily low time data will be saved in folder: 'daily_low_times_data'
Processing 1 ticker(s) from 2025-01-01 to 2025-04-26 at 1h interval.

📦 Processing batch 1 of 1
   Processing daily lows for 3MINDIA.NS...
❌ Error processing 3MINDIA.NS: Cannot index with multidimensional key

🏁 Done! Daily low times CSV files saved in folder: 'daily_low_times_data'
